In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load Excel sheets
dfs = pd.read_excel('../data/2022_Gravity.xlsx', sheet_name=None)

# Gravity correction function
def process_gravity(df):
    # Drift Correction
    base_idx = np.where(df['Base'] == 'base')[0]
    times = df['Time (h)']
    obs = df['obs (mGal)']
    g_drift = np.interp(times, times[base_idx], obs[base_idx])
    drift = obs - g_drift

    # Free-air Correction
    fa_corr = -0.3086 * df["Elevation (m)"]
    fa_corrected = drift - fa_corr

    # Bouguer Correction (assumes 2670 kg/m³)
    bouguer_corr = 0.0921995 * df["Elevation (m)"]
    final_corr = fa_corrected - bouguer_corr

    # Store in DataFrame
    df["Corrected"] = final_corr
    return df

# Apply to all sheets
for sheet in dfs:
    dfs[sheet] = process_gravity(dfs[sheet])

# Save corrected results
with pd.ExcelWriter("../data/2022_Gravity_Corrected.xlsx") as writer:
    for sheet_name, df in dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
